In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import optuna

/home/carolima/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v11.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v11.csv")



In [4]:
# Separating features and target variable
X = challenge_set_updated.drop('tow', axis=1)
y = challenge_set_updated['tow']

In [7]:
# Dropping unnecessary columns
to_drop = [
    'offblock_to_arrival_duration', 'normalized_taxi_ratio', 'MALW_kg', 'wind_distance_ARR_100', 
    'average_airspeed_ARR_100', 'track_variation_ARR_100', 'is_offblock_weekend', 'Num_Engines', 
    'flown_distance_ARR_100', 'average_humidity_ARR_100', 'average_temperature_ARR_100', 
    'arrival_minute', 'track_variation_ENR', 'groundspeed_ARR_100', 'average_vertical_rate_ARR_100', 
    'taxiout_time', 'offblock_minute', 
    'average_airspeed_ENR', 'specific_energy_ENR', 'taxi_ratio',  
    'specific_energy_ARR_100', 'is_offblock_rush_hour', 'wind_distance_ENR', 'groundspeed_ENR', 
    'altitude_difference', 'average_vertical_rate_ENR', 'bearing', 'Altitude_ades'
]


# Function to filter out non-existing columns before dropping
def filter_existing_columns(df, columns_to_drop):
    return [col for col in columns_to_drop if col in df.columns]

# Check the columns in the DataFrame X
print("Current Columns in X:", X.columns.tolist())

# Filter the columns to drop
existing_columns_to_drop = filter_existing_columns(X, to_drop)

# Drop the unnecessary columns
X.drop(existing_columns_to_drop, axis=1, inplace=True)


Current Columns in X: ['taxiout_time', 'flown_distance', 'track_variation_ARR_100', 'track_variation_DEP_40', 'track_variation_ENR', 'average_vertical_rate_ARR_100', 'average_vertical_rate_DEP_40', 'average_vertical_rate_ENR', 'average_airspeed_ARR_100', 'average_airspeed_DEP_40', 'average_airspeed_ENR', 'groundspeed_ARR_100', 'groundspeed_DEP_40', 'groundspeed_ENR', 'wind_distance_ARR_100', 'wind_distance_DEP_40', 'wind_distance_ENR', 'average_temperature_ARR_100', 'average_temperature_DEP_40', 'average_temperature_ENR', 'average_humidity_ARR_100', 'average_humidity_DEP_40', 'average_humidity_ENR', 'specific_energy_ARR_100', 'specific_energy_DEP_40', 'specific_energy_ENR', 'flown_distance_ARR_100', 'flown_distance_DEP_40', 'flown_distance_ENR', 'average_altitude_ARR_100', 'average_altitude_DEP_40', 'average_altitude_ENR', 'specific_energy', 'taxi_ratio', 'flight_speed', 'normalized_taxi_ratio', 'speed_per_distance', 'cumulative_avg_speed', 'airspeed_specific_energy_ARR', 'airspeed_spe

In [8]:
cat_names = [
    'adep', 'ades', 'aircraft_type', 'wtc', 'airline', 'offblock_hour', 'offblock_minute', 
    'offblock_day_of_week', 'offblock_month', 'offblock_week_of_year', 'offblock_season', 
    'arrival_hour', 'arrival_minute', 'is_offblock_weekend', 'is_offblock_rush_hour', 
    'flight_duration_category', 'adep_region', 'ades_region', 'same_country_flight', 
    'same_region_flight', 'flight_direction', 'is_intercontinental', 'Manufacturer', 
    'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight', 'adep_geo_cluster', 'ades_geo_cluster'
]

# Filtering categorical features that are still present after dropping columns
selected_cat_names = [x for x in cat_names if x not in to_drop]

In [10]:
from sklearn.preprocessing import LabelEncoder

# Check available columns in X
print("Current Columns in X:", X.columns.tolist())

# Handle categorical columns by Label Encoding
label_encoder = LabelEncoder()

# Specify the columns that are categorical
categorical_cols = ['adep', 'ades', 'aircraft_type', 'wtc', 'airline', 'offblock_season', 
                    'flight_duration_category', 'adep_region', 'ades_region', 'flight_direction', 
                    'Manufacturer', 'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight']

# Filter out only those categorical columns that exist in X
existing_categorical_cols = [col for col in categorical_cols if col in X.columns]

# Convert these categorical columns into numerical form using Label Encoding
for col in existing_categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Now, you can proceed with training LightGBM with this processed data


Current Columns in X: ['flown_distance', 'track_variation_DEP_40', 'average_vertical_rate_DEP_40', 'average_airspeed_DEP_40', 'groundspeed_DEP_40', 'wind_distance_DEP_40', 'average_temperature_DEP_40', 'average_temperature_ENR', 'average_humidity_DEP_40', 'average_humidity_ENR', 'specific_energy_DEP_40', 'flown_distance_DEP_40', 'flown_distance_ENR', 'average_altitude_ARR_100', 'average_altitude_DEP_40', 'average_altitude_ENR', 'specific_energy', 'flight_speed', 'speed_per_distance', 'cumulative_avg_speed', 'airspeed_specific_energy_ARR', 'airspeed_specific_energy_DEP', 'airspeed_specific_energy_ENR', 'vertical_rate_airspeed_ratio_ARR', 'vertical_rate_airspeed_ratio_DEP', 'Approach_Speed_knot', 'Wingspan_ft_without_winglets_sharklets', 'Wingspan_ft_with_winglets_sharklets', 'Length_ft', 'Tail_Height_at_OEW_ft', 'Wheelbase_ft', 'Cockpit_to_Main_Gear_ft', 'Main_Gear_Width_ft', 'Parking_Area_ft2', 'Total_IFR_Operations_2021_2022', 'MTOW_kg', 'Latitude_adep', 'Longitude_adep', 'Altitude_ad

In [11]:
# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical feature indices for LightGBM
cat_feature_indices = [X_train.columns.get_loc(col) for col in selected_cat_names if col in X_train.columns]

# Define the objective function for hyperparameter tuning with Optuna
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 0.2),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.0, 10.0),
        'device_type': 'gpu',  # Use GPU for training

    }

    model = lgb.LGBMRegressor(**params, n_estimators=5000)

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
        categorical_feature=cat_feature_indices
    )
    
    val_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, val_pred, squared=False)
    
    return rmse

In [12]:
# Create the Optuna study and start optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# Display the best hyperparameters found
print(f"Best trial: {study.best_trial.params}")


[I 2024-09-26 09:47:09,692] A new study created in memory with name: no-name-2a58bf00-646a-40d2-b24f-aeb3c329794b


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[4998]	valid_0's rmse: 2762.88


/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-09-26 09:47:49,648] Trial 0 finished with value: 2762.8829082473276 and parameters: {'learning_rate': 0.035615551544055386, 'num_leaves': 2059, 'max_depth': 8, 'min_data_in_leaf': 77, 'feature_fraction': 0.5785152976273922, 'bagging_fraction': 0.9296503790920617, 'lambda_l1': 5.874308412280258, 'lambda_l2': 7.256133483225944, 'min_gain_to_split': 0.035909932886852784, 'min_child_weight': 3.2229204107038543}. Best is trial 0 with value: 2762.8829082473276.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[5000]	valid_0's rmse: 2788.2


/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-09-26 09:48:17,882] Trial 1 finished with value: 2788.1957382239652 and parameters: {'learning_rate': 0.03326177130240085, 'num_leaves': 1252, 'max_depth': 7, 'min_data_in_leaf': 69, 'feature_fraction': 0.6719196873411559, 'bagging_fraction': 0.6537943478041194, 'lambda_l1': 0.04541017854773233, 'lambda_l2': 0.23878889991223673, 'min_gain_to_split': 0.18061753212130904, 'min_child_weight': 6.765029338322633}. Best is trial 0 with value: 2762.8829082473276.


...

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[4998]	valid_0's rmse: 2736.22


/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-09-26 10:21:35,632] Trial 28 finished with value: 2736.215953760433 and parameters: {'learning_rate': 0.01781140506372878, 'num_leaves': 676, 'max_depth': 13, 'min_data_in_leaf': 37, 'feature_fraction': 0.9032931764855036, 'bagging_fraction': 0.5957893744650885, 'lambda_l1': 3.4431471917350724, 'lambda_l2': 6.864105933603034, 'min_gain_to_split': 0.09310943475057917, 'min_child_weight': 4.190548190460737}. Best is trial 22 with value: 2725.1024902499903.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[5000]	valid_0's rmse: 2788.1


/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-09-26 10:22:06,022] Trial 29 finished with value: 2788.103937955437 and parameters: {'learning_rate': 0.03428774108057072, 'num_leaves': 1059, 'max_depth': 7, 'min_data_in_leaf': 53, 'feature_fraction': 0.7701605715402617, 'bagging_fraction': 0.6703326119680071, 'lambda_l1': 4.856842525798516, 'lambda_l2': 9.952080142998154, 'min_gain_to_split': 0.0316325009866814, 'min_child_weight': 2.767631378386426}. Best is trial 22 with value: 2725.1024902499903.


Best trial: {'learning_rate': 0.040232447540238, 'num_leaves': 1035, 'max_depth': 10, 'min_data_in_leaf': 35, 'feature_fraction': 0.8880567898770956, 'bagging_fraction': 0.7109490701551315, 'lambda_l1': 6.156991762243389, 'lambda_l2': 7.71117214126504, 'min_gain_to_split': 0.051274563395863657, 'min_child_weight': 4.021845555499864}


In [13]:
# Train the final LightGBM model with the best hyperparameters
best_params = study.best_trial.params

best_model = lgb.LGBMRegressor(**best_params, n_estimators=10000)

best_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
  #  early_stopping_rounds=50,
    categorical_feature=cat_feature_indices
)



LGBMRegressor(bagging_fraction=0.7109490701551315,
              feature_fraction=0.8880567898770956, lambda_l1=6.156991762243389,
              lambda_l2=7.71117214126504, learning_rate=0.040232447540238,
              max_depth=10, min_child_weight=4.021845555499864,
              min_data_in_leaf=35, min_gain_to_split=0.051274563395863657,
              n_estimators=10000, num_leaves=1035)

In [14]:
# Inspect the model's feature names (from the training phase)
model_features = best_model.feature_name_  # Or check best_model.booster_.feature_name() for lightgbm

# Print model's features and submission_set columns for debugging
print("Model features:", model_features)
print("Submission set columns:", submission_set.columns.tolist())

# Ensure the submission_set has the same features as the model
# Add missing columns with default values (e.g., 0 or the mean)
for feature in model_features:
    if feature not in submission_set.columns:
        submission_set[feature] = 0  # You can choose a more appropriate fill value


Model features: ['flown_distance', 'track_variation_DEP_40', 'average_vertical_rate_DEP_40', 'average_airspeed_DEP_40', 'groundspeed_DEP_40', 'wind_distance_DEP_40', 'average_temperature_DEP_40', 'average_temperature_ENR', 'average_humidity_DEP_40', 'average_humidity_ENR', 'specific_energy_DEP_40', 'flown_distance_DEP_40', 'flown_distance_ENR', 'average_altitude_ARR_100', 'average_altitude_DEP_40', 'average_altitude_ENR', 'specific_energy', 'flight_speed', 'speed_per_distance', 'cumulative_avg_speed', 'airspeed_specific_energy_ARR', 'airspeed_specific_energy_DEP', 'airspeed_specific_energy_ENR', 'vertical_rate_airspeed_ratio_ARR', 'vertical_rate_airspeed_ratio_DEP', 'Approach_Speed_knot', 'Wingspan_ft_without_winglets_sharklets', 'Wingspan_ft_with_winglets_sharklets', 'Length_ft', 'Tail_Height_at_OEW_ft', 'Wheelbase_ft', 'Cockpit_to_Main_Gear_ft', 'Main_Gear_Width_ft', 'Parking_Area_ft2', 'Total_IFR_Operations_2021_2022', 'MTOW_kg', 'Latitude_adep', 'Longitude_adep', 'Altitude_adep', '

/tmp/ipykernel_826648/106421064.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  submission_set[feature] = 0  # You can choose a more appropriate fill value
/tmp/ipykernel_826648/106421064.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  submission_set[feature] = 0  # You can choose a more appropriate fill value
/tmp/ipykernel_826648/106421064.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

In [15]:
from sklearn.preprocessing import LabelEncoder

# List of categorical columns that need to be converted to numeric
categorical_columns = ['adep', 'ades', 'aircraft_type', 'wtc', 'airline']

# Initialize LabelEncoder
label_encoders = {}

# Convert each categorical column to numeric using LabelEncoder
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    submission_set[col] = label_encoders[col].fit_transform(submission_set[col].astype(str))

# Print to check the converted types
print("Submission set dtypes after label encoding:", submission_set.dtypes)

# Ensure the submission_set has the same features as the model
submission_set_features = submission_set[model_features]

# Predict using the aligned features
submission_set['tow'] = best_model.predict(submission_set_features)

# Inspect the result
print(submission_set.head())


Submission set dtypes after label encoding: flight_id               int64
date                   object
callsign               object
adep                    int64
name_adep              object
                        ...  
ades_geo_cluster_15     int64
ades_geo_cluster_16     int64
ades_geo_cluster_17     int64
ades_geo_cluster_18     int64
ades_geo_cluster_19     int64
Length: 292, dtype: object
   flight_id        date                          callsign  adep  \
0  248753821  2022-01-01  3b3de0f3ad0ee192513995c02f7bf7cf   331   
1  248753822  2022-01-01  e06dd03d4a879ca37d9e18c1bd7cad16    11   
2  248754498  2022-01-01  2d3b1c962c78c4ebeef11bcd51b9e94c   158   
3  248757623  2022-01-01  81564432d3ee97c4bdf4cd8f006753dc    45   
4  248763603  2022-01-01  84be079d7e660db105d91f600b4b3d59    74   

                name_adep country_code_adep  ades        name_ades  \
0  Istanbul Sabiha Gokcen                TR   185             Lyon   
1                Brussels                BE   138 

In [16]:

from datetime import datetime
import pytz

saopaulo_tz = pytz.timezone('America/Sao_Paulo')

timestamp = datetime.now(saopaulo_tz).strftime('%Y%m%d_%H%M%S')


In [17]:

import os

# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")
submission_set.to_csv(submission_file, index=False)